In [0]:
%sql
/* 03_silver_enc_demographics_join.ipynb
SOURCE:  Snapshot join of Silver encounters and Silver patients for demographic enrichment.
OUTPUT: `kardia_silver.silver_encounters_demographics`, refreshed as a batch table.
PATTERN: Snapshot left join
TRIGGER: CTAS; rerun after each encounter load or on schedule */

-- 1. Ensure database exists
CREATE DATABASE IF NOT EXISTS kardia_silver;

-- 2. Create hourly metrics summarizing claim volume per ingestion hour
CREATE OR REPLACE TABLE kardia_silver.silver_encounters_demographics AS
    WITH patients_clean AS (
        SELECT ID,
               CASE WHEN GENDER IN ('M','F') THEN GENDER END AS GENDER,
               CASE WHEN BIRTH_YEAR BETWEEN 1900 AND YEAR(CURRENT_DATE()) THEN BIRTH_YEAR END AS BIRTH_YEAR
        FROM kardia_silver.silver_patients
    )
    SELECT e.EncounterID,
           e.PatientID,
           e.START_TS,
           e.CODE,
           e.DESCRIPTION,
           e.REASONCODE,
           e.REASONDESCRIPTION,
           p.GENDER,
           p.BIRTH_YEAR
    FROM kardia_silver.silver_encounters AS e
    LEFT JOIN patients_clean AS p
        ON e.PatientID = p.ID;